## Carbon Sequestration Map API Query Demo
This demo allows the user to query our predicted map of carbon sequestration rates. Given an area of interest, the API will return summary statistics of the region including mean, min, max, and count. All units are in megagrams of carbon per hectare per year (Mg C/ha/yr).

Our model is a prediction map of potential carbon sequestration rates for young, naturally regenerating forests at about 1 kilometer resolution. The map was made by training a Random Forest based machine learning model on forest inventory data using about 80 covariates, with variables covering soil, topography, and climate. 

This map and our API can be used by governments, land use planners, and restoration organizations to assess the potential carbon sequestration potential from naturally regenerating forests in a given region.

This API was put together by the World Resources Institute.

In [1]:
#Import needed libraries
#If you do not have the necessary library, uncomment the corresponding line to install via pip
#!pip install fiona
#!pip install requests
#!pip install zipfile
import os
import io
import sys
import requests
import fiona
import zipfile
%autosave 0


Autosave disabled


In [2]:
#To use this module, you need to have a shapefile to perform calculations over.
#If you do not have one prepared, you can run this module to download and extract a shapefile

r = requests.get('https://gfw-files.s3.amazonaws.com/ai4e/Appalachian_forest.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path='~/') # extract to folder
filenames =  z.namelist()

shapefile = [x for x in filenames if '.shp' in x][0]
path_to_shapefile = os.path.join('~/',shapefile)
print('Copy and paste this file in the form below: {}'.format(path_to_shapefile))

Copy and paste this file in the form below: ~/Appalachian_forest.shp


In [3]:
#Use the form below to enter the path to your shapefile or geojson. This file should be on your local computer
#For example: /Users/kristine/WRI/MachineLearning/CarbonAI/ZonalStats/Appalachian_shape/Appalachian_shape.shp
path_to_shapefile = input("Enter the path to your shapefile: ") 
print('The following input file was selected: {}\n'.format(path_to_shapefile))

#First check if the shapefile has any errors:    
try:
    src = fiona.open(path_to_shapefile)
    print('Inputted shapefile is okay!')
except:
    print('Inputted shapefile is not formatted correctly and has erroneous geometries. Please fix the geometries and try again.')
    

Enter the path to your shapefile: ~/Appalachian_forest.shp
The following input file was selected: ~/Appalachian_forest.shp

Inputted shapefile is okay!


In [4]:
#Define API querying packages
def get_my_api(URL):
    
    try:           
        r = requests.get(URL)
        if r.ok:
            return r.content.decode('utf-8')
        else:
            return None

    except Exception as e:
        print('Exception running API: {}'.format(e))
        
def post_my_api(URL,myobj):
    x = requests.post(URL, json = myobj)
    return x.content.decode('utf-8')


In [5]:
#Check that the API is running!
result = get_my_api('http://40.90.246.141/v1/my_api/tasker/echo/yes')
print('Is it running?')
print(result)

Is it running?
Echo: yes


In [6]:
#Process the shapefile to convert it to a json to send to API
print('The following file was selected: {}'.format(path_to_shapefile))
features = []
with fiona.open(path_to_shapefile) as src:
    for f in src:
        features.append(f)
content = {'features':features}

#Post query to API
task_id_string = post_my_api('http://40.90.246.141/v1/my_api/tasker/example',content)
task_id = int(task_id_string.split(':')[1])

#Get information on task
result = get_my_api('http://40.90.246.141/v1/my_api/tasker/task/{}'.format(task_id))
print(result)


The following file was selected: ~/Appalachian_forest.shp
{"TaskId": 3384, "Status": "Writing to geojson", "Timestamp": "2019-11-01 14:29:14", "Endpoint": "uri"}



In [8]:
#Run this cell to get the status of your query!
result = get_my_api('http://40.90.246.141/v1/my_api/tasker/task/{}'.format(task_id))
print(result)

{"TaskId": 3384, "Status": "Completed: Here are the statistics for your region(s): [{'min': 1.2026921510696411, 'max': 2.526801586151123, 'mean': 1.543001412508183, 'count': 268848}]", "Timestamp": "2019-11-01 14:29:14", "Endpoint": "uri"}

